In [1]:
class ScratchSVMClassifier():
    """
    Parameters
    ----------
    max_iter : int
      イテレーション回数
    lr : float
      学習率
    kernel_type : str , default='linear' 
      カーネルの設定 
    
    
    Attributes
    ----------
    kernel : 次の2形のndarray (n_samples, n_samples)
      kernelのi,j行列
    lagrangian: float
      ラグランジュ乗数 lambda
    
    """
    
    def __init__(self,max_iter, lr, kernel_type='linear'):
        # parameterを属性として保持
        self.max_iter = max_iter
        self.lr = lr
        self.kernel_type = kernel_type
        self.lagrangian = 0
        self.index_array = 0
        
    def make_kernel(self, A, B):
        if self.kernel_type == 'linear':
            kernel = np.dot(A.T, B)
        return kernel.T
     
    def update_lambda(self, X, y):

        y_i_j = np.dot(y.T, y)

        # kernelの呼び出し
        kernel = self.make_kernel(X, X)
        # y_i_jとk(xi, xj)のアダマール積 どちらも(n_samples, n_samples)
        y_i_j_k = y_i_j*kernel
        # λ(1,n_samples) と y_i_j_k(n_samples, n_samples)の積 shape(1, n_samples)
        update_param = np.dot(self.lagrangian, y_i_j_k)
        self.lagrangian = self.lagrangian + self.lr*(1-update_param)
        
    def fit(self, X, y):
        # Xとyを転置する
        X = X.T
        y = y.reshape(1,-1)
        
         # 閾値
        threshold = 1e-5
        
        # lambdaの初期値を設定する 次の形のndarray np.array(1, n_samples)
        self.lagrangian = (np.random.rand(X.shape[1])).reshape(1,-1)
        print(self.lagrangian)
        
        for i in range(self.max_iter):
            # λの更新
            self.update_lambda(X, y)
            # 負のλを0に
            self.lagrangian = np.where(self.lagrangian > 0 ,self.lagrangian, 0)
            # 閾値を超えたλのインデックスのxをサポートベクトルに
            index_array = np.where(self.lagrangian>threshold)[1]
            
        self.index_array = index_array
        # n個のサポートベクターの行列  ndarray(n_features, n)      
        self.support_vectors = X[:,index_array]
        # supportvector の正解ラベルを属性として格納
        self.y_support_vector_label = y[0,self.index_array]
        
    def predict(self, X_test):
        X_test = X_test.T

        ks = self.make_kernel(X_test, self.support_vectors)
        
        support_vector_lagrangian = self.lagrangian[0, self.index_array].reshape(1,-1)
        
        lambda_y = support_vector_lagrangian*self.y_support_vector_label
        y_pred = np.dot(lambda_y, ks)

        # 要素が負の場合-1にし、正の場合1にする
        y_pred = np.where(y_pred<0, -1, y_pred)
        y_pred = np.where(y_pred>0, 1, y_pred)
        return y_pred
    
    def score(self, X_test,y_train, y_test):
        y_pred = self.predict(X_test, y_train).reshape(-1,)
        # 配列を比較してnp.sumする
        score = np.sum(y_pred == y_test) / len(y_test)
        return score